In [19]:
import os
import nltk
import pandas as pd
import numpy as np
import gensim.downloader
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize 

import syllables 
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

from eval import evaluate

In [2]:
word_vector = gensim.downloader.load('word2vec-google-news-300')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/nlp_project

# !pip install syllables

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ankit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def add_single_features(name,isTrain) :
    df = pd.read_csv("data/preprocessed/" +name )

    frequency_of_word = {}
    for i in range(len(df)) :
        word = df.at[i,"token"]
        if word in frequency_of_word :
            frequency_of_word[word]+=1
        else :
            frequency_of_word[word] = 1

    def vowel_count(str): 
        count = 0
        vowel = set("aeiouAEIOU") 
        for alphabet in str: 
            if alphabet in vowel: 
                  count = count + 1
        return count 


    def contain_numeral(str) :
        num = "0123456789"
        for alphabet in str: 
            if alphabet in num: 
                return True
        return False

    token_length=[]
    token_syllable= []
    token_vowels = []
    token_frequency = []
    token_contain_numeral = []
    nearest_class = []
    
    for i in range(len(df)) :
        word = df.at[i,"token"]
        if type(word) is not str :
            token_length.append(0)
            token_syllable.append(0)
            token_frequency.append(0)
            token_vowels.append(0)
            token_contain_numeral.append(False)
            if isTrain :
                nearest_class.append(round(4*df.at[i,"complexity"])+1)   

            continue
        token_length.append(len(word))
        token_syllable.append(syllables.estimate(word))
        token_vowels.append(vowel_count(word))
        token_frequency.append(frequency_of_word[word])
        token_contain_numeral.append(contain_numeral(word))

        # print(type(df.at[i,"complexity"]))
        if isTrain :
            nearest_class.append(round(4*df.at[i,"complexity"])+1)

    df['token_length'] = token_length
    df['token_syllable'] = token_syllable
    df['token_vowels'] = token_vowels
    df['token_frequency'] = token_frequency
    df['token_contain_numeral'] = token_contain_numeral
    if isTrain :
        df['nearest_class'] = nearest_class
    df.to_csv("data/added_features/"+name,index=False)

In [8]:
add_single_features("lcp_single_test_preprocessed.csv",False)
add_single_features("lcp_single_train_preprocessed.csv",True)
add_single_features("lcp_multi_test_preprocessed.csv",False)
add_single_features("lcp_multi_train_preprocessed.csv",True)

###  models on single

In [9]:
data_folder = "data/added_features"
df_Train = pd.read_csv(os.path.join(data_folder,"lcp_single_train_preprocessed.csv"),index_col=0)
df_Test = pd.read_csv(os.path.join(data_folder,"lcp_single_test_preprocessed.csv"),index_col=0)
df_Test.head()

,corpus,sentence,token,token_length,token_syllable,token_vowels,token_frequency,token_contain_numeral
id,,,,,,,,
39TX062QX1OHFOH8FUL76K5L7D3X3S,bible,speak much prince world come nothing,prince,6,2,2,5,False
3CIS7GGG65JS8I3AZ9RG54AE4MUUEA,bible,house shall turned others field wife together ...,inhabitant,10,4,4,3,False
379OL9DBSSESUVWY1Z8JGBFG9BTY92,bible,stranger terrible nation cut left mountain val...,bough,5,1,2,2,False
3DFYDSXB2W00JYP2DA272KN69UQUJV,bible,sharpen tongue like sword aim arrow deadly word,arrow,5,2,2,3,False
31YWE12TE0CZG7IVH6OXJ1H1CFPX7X,bible,obey leader submit watch behalf soul give acco...,account,7,2,3,4,False


In [10]:
stat_feature = np.array(df_Train[['token_length','token_syllable','token_vowels','token_frequency','token_contain_numeral']])

embed_word = np.array(list(df_Train['token'].apply(lambda x:word_vector[x] if x in word_vector else word_vector['unk'])))
embed_sentence = np.array(list(df_Train['sentence'].apply
    (
    lambda x:
    sum([word_vector[w] if w in word_vector else word_vector['unk'] for w in x.split()])/len(x.split())
    )
                              )
                         )
Train_Vector = np.hstack((stat_feature,.5*embed_word+.5*embed_sentence))
Train_Vector.shape

(7662, 305)

In [11]:
stat_feature = np.array(df_Test[['token_length','token_syllable','token_vowels','token_frequency','token_contain_numeral']])

embed_word = np.array(list(df_Test['token'].apply(lambda x:word_vector[x] if x in word_vector else word_vector['unk'])))
embed_sentence = np.array(list(df_Test['sentence'].apply
    (
    lambda x:
    sum([word_vector[w] if w in word_vector else word_vector['unk'] for w in x.split()])/len(x.split())
    )
                              )
                         )
Test_Vector = np.hstack((stat_feature,.5*embed_word+.5*embed_sentence))
Test_Vector.shape

(917, 305)

In [12]:
submission_folder = "predictions/single_with_features"

In [14]:
# Linear Regression
reg = LinearRegression().fit(Train_Vector, np.array(df_Train['complexity']))
y_pred = reg.predict(Test_Vector)

pred = pd.DataFrame({"ID":df_Test.index, "complexity":y_pred})
pred.to_csv(submission_folder+"/linear_regression_with_features.csv", index=False, header=False)

In [15]:
# SVM regressor
reg = SVR().fit(Train_Vector, np.array(df_Train['complexity']))
y_pred = reg.predict(Test_Vector)

pred = pd.DataFrame({"ID":df_Test.index, "complexity":y_pred})
pred.to_csv(submission_folder+"/SVM_with_features.csv", index=False, header=False)

In [16]:
# Gradient Boosting
reg = GradientBoostingRegressor().fit(Train_Vector, np.array(df_Train['complexity']))
y_pred = reg.predict(Test_Vector)

pred = pd.DataFrame({"ID":df_Test.index, "complexity":y_pred})
pred.to_csv(submission_folder+"/gradient_boosting_with_features.csv", index=False, header=False)

In [17]:
# MLP Regressor
regr = MLPRegressor(hidden_layer_sizes=(150)).fit(Train_Vector, np.array(df_Train['complexity']))
y_pred = reg.predict(Test_Vector)

pred = pd.DataFrame({"ID":df_Test.index, "complexity":y_pred})
pred.to_csv(submission_folder+"/MLP_with_features.csv", index=False, header=False)

### evaluate baseline models

In [20]:
evaluate(submission_folder, "references/lcp_single_test_labelled_preprocessed.csv")


For file MLP_with_features.csv
pearson  :  0.6510099362950927
spearman :  0.6197440617933732
mae      :  0.07254862857837216
mse      :  0.009343434124671623
r2       :  0.4227129699873945

For file linear_regression_with_features.csv
pearson  :  0.6311508461209863
spearman :  0.6077421054566633
mae      :  0.07661968054084875
mse      :  0.009917703426107611
r2       :  0.3872315597232592

For file gradient_boosting_with_features.csv
pearson  :  0.6510099362950927
spearman :  0.6197440617933732
mae      :  0.07254862857837216
mse      :  0.009343434124671623
r2       :  0.4227129699873945

For file SVM_with_features.csv
pearson  :  0.6744168783485909
spearman :  0.6307686935582127
mae      :  0.07273162781364834
mse      :  0.008875871431429068
r2       :  0.4516014787439301
